In [ ]:
!pip install python-constraint

In [ ]:

from constraint import Problem, AllDifferentConstraint
from collections import defaultdict
import pandas as pd

def parse_time(t):
    from datetime import datetime
    return datetime.strptime(t, "%H:%M")

def get_time_of_day(start_time):
    hour = int(start_time.split(":")[0])
    return "matin" if hour < 12 else "apres-midi"


In [ ]:

def validate_data_consistency(employees, business_schedule, required_employees_per_day):
    print("\n🔍 Vérification de la cohérence des données...\n")
    errors = []
    jours_connus = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi", "Samedi", "Dimanche"]

    for name, data in employees.items():
        for day, _ in data.get("assigned_days", []):
            if day not in business_schedule or not business_schedule[day]:
                errors.append(f"❌ {name} est assigné à un jour fermé ({day}).")

        for key in ["assigned_days", "days_off", "vacation_days"]:
            for entry in data.get(key, []):
                day = entry if isinstance(entry, str) else entry[0]
                if day not in jours_connus:
                    errors.append(f"❌ {name} a un jour inconnu dans {key} : '{day}'.")

    for day, required in required_employees_per_day.items():
        available = sum(
            1 for name, data in employees.items()
            if day not in [d if isinstance(d, str) else d[0] for d in data.get("days_off", [])]
        )
        if available < required:
            errors.append(f"❌ Pas assez d'employés disponibles le {day} : {available} dispo(s), {required} requis.")

    all_shifts = []
    for day, intervals in business_schedule.items():
        nb_required = required_employees_per_day.get(day, 1)
        for start, end in intervals:
            duration = (parse_time(end) - parse_time(start)).total_seconds() / 3600
            all_shifts.append(duration * nb_required)

    total_needed = sum(all_shifts)
    total_available = sum(data["weekly_hours"] for data in employees.values())
    if total_needed > total_available:
        errors.append(f"❌ Heures à couvrir = {total_needed}h, capacité totale = {total_available}h → insuffisante.")

    if errors:
        for e in errors:
            print(e)
        return False
    else:
        print("✅ Données cohérentes. Prêt pour exécution.\n")
        return True


In [ ]:

def generate_multi_shifts_with_moments(business_schedule, required_employees_per_day):
    shifts = []
    for day, intervals in business_schedule.items():
        nb_required = required_employees_per_day.get(day, 1)
        for start, end in intervals:
            duration = (parse_time(end) - parse_time(start)).total_seconds() / 3600
            moment = get_time_of_day(start)
            for i in range(nb_required):
                shifts.append({
                    "var_name": f"{day}_{start}_{end}_poste{i+1}",
                    "day": day,
                    "start": start,
                    "end": end,
                    "duration_hours": duration,
                    "position": i+1,
                    "moment": moment
                })
    return shifts


In [ ]:

def build_multi_scheduler(employees, shifts):
    employee_names = list(employees.keys())
    problem = Problem()

    for shift in shifts:
        day, moment = shift["day"], shift["moment"]
        available_employees = [
            e for e in employee_names
            if (day, moment) not in employees[e].get("vacation_days", []) and
               day not in [d if isinstance(d, str) else d[0] for d in employees[e].get("days_off", [])]
        ]
        problem.addVariable(shift["var_name"], available_employees)

    from constraint import AllDifferentConstraint
    shift_groups = defaultdict(list)
    for shift in shifts:
        key = (shift["day"], shift["start"], shift["end"])
        shift_groups[key].append(shift["var_name"])
    for group in shift_groups.values():
        problem.addConstraint(AllDifferentConstraint(), group)

    def respect_quota(*args):
        total_by_employee = {e: 0 for e in employee_names}
        for assigned, shift in zip(args, shifts):
            total_by_employee[assigned] += shift["duration_hours"]
        return all(total_by_employee[e] <= employees[e]["weekly_hours"] for e in employees)

    def respect_assigned_days(*args):
        work_by_employee = defaultdict(set)
        for assigned, shift in zip(args, shifts):
            work_by_employee[assigned].add((shift["day"], shift["moment"]))
        for e in employee_names:
            for d in employees[e].get("assigned_days", []):
                if d not in work_by_employee[e]:
                    return False
        return True

    var_names = [s["var_name"] for s in shifts]
    problem.addConstraint(respect_quota, var_names)
    problem.addConstraint(respect_assigned_days, var_names)

    return problem.getSolutions(), shifts


In [ ]:

def select_best_balanced_solution(solutions, shifts, employees, tolerance_ratio=0.25):
    def total_hours(solution):
        total = defaultdict(float)
        for shift in shifts:
            emp = solution[shift["var_name"]]
            total[emp] += shift["duration_hours"]
        return total

    filtered = [
        s for s in solutions
        if all(abs(total_hours(s)[e] - employees[e]["weekly_hours"]) <= tolerance_ratio * employees[e]["weekly_hours"]
               for e in employees)
    ]

    def score(sol):
        hours = total_hours(sol)
        return sum(abs(hours[e] - employees[e]["weekly_hours"]) for e in employees)

    return min(filtered, key=score) if filtered else min(solutions, key=score)


In [ ]:

# --- Données d'entrée ---

business_schedule = {
    "Lundi": [("08:00", "12:00"), ("14:00", "18:00")],
    "Mardi": [("08:00", "12:00"), ("14:00", "18:00")],
    "Mercredi": [("08:00", "12:00")],
    "Jeudi": [("08:00", "12:00"), ("14:00", "18:00")],
    "Vendredi": [("08:00", "12:00"), ("14:00", "18:00")],
    "Samedi": [("09:00", "13:00")],
    "Dimanche": []
}

employees = {
    "Alice": {
        "weekly_hours": 20,
        "days_off": [],
        "vacation_days": [("Mercredi", "matin")],
        "assigned_days": [("Lundi", "matin"), ("Vendredi", "apres-midi")]
    },
    "Bob": {
        "weekly_hours": 30,
        "days_off": ["Jeudi"],
        "vacation_days": [],
        "assigned_days": [("Mardi", "matin")]
    },
    "Charlie": {
        "weekly_hours": 40,
        "days_off": [],
        "vacation_days": [],
        "assigned_days": []
    }
}

required_employees_per_day = {
    "Lundi": 2,
    "Mardi": 2,
    "Mercredi": 1,
    "Jeudi": 2,
    "Vendredi": 2,
    "Samedi": 1
}

if validate_data_consistency(employees, business_schedule, required_employees_per_day):
    shifts = generate_multi_shifts_with_moments(business_schedule, required_employees_per_day)
    solutions, shifts = build_multi_scheduler(employees, shifts)

    if solutions:
        best_solution = select_best_balanced_solution(solutions, shifts, employees)

        planning = []
        hours_counter = defaultdict(float)

        for shift in shifts:
            assigned = best_solution[shift["var_name"]]
            hours_counter[assigned] += shift["duration_hours"]
            planning.append({
                "Jour": shift["day"],
                "Début": shift["start"],
                "Fin": shift["end"],
                "Poste #": shift["position"],
                "Moment": shift["moment"],
                "Employé assigné": assigned,
                "Durée (h)": shift["duration_hours"]
            })

        df_planning = pd.DataFrame(planning)
        df_report = pd.DataFrame([
            {"Employé": emp, "Heures assignées": hours_counter[emp], "Heures contrat": employees[emp]["weekly_hours"]}
            for emp in employees
        ])

        from IPython.display import display
        display(df_planning)
        display(df_report)

        df_planning.to_csv("planning.csv", index=False)
        df_report.to_csv("bilan_employes.csv", index=False)
    else:
        print("❌ Aucune solution valide trouvée.")
else:
    print("🛑 Résolution annulée à cause d'incohérences dans les données.")
